In [26]:
import pandas as pd
import numpy as np

from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, Embedding, Input, Dropout, Activation, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [27]:
dataset = 'C://Users//H//Desktop//Spring18//-AdvancedDatabases//Project//Project_Work//final_dataset.csv'
data = pd.read_csv(dataset, encoding='utf-8-sig')
data.head()

,Tweet,For guns?,Location,For guns,Against guns,Makes no sense
0,@Mary_rnntt2 @lauren_hoggs You're blinded by y...,0.0,"Mountain Lakes, NJ",0,1,0
1,"Hey #MNLEG, don’t be idiotic. gun laws, not pa...",0.0,"Minneapolis, MN",0,1,0
2,"Gun owners, women, kids, all opposed to the ab...",0.0,"Houston, TX",0,1,0
3,@PhilipRucker Is he taking guns away from ment...,0.0,"Broomall, PA",0,1,0
4,Blissfully unaware that it’s too late: Califor...,1.0,"Teaneck, NJ",1,0,0


In [28]:
training = data.iloc[:340]
testing = data.iloc[340:]

In [42]:
labels = ['For guns', 'Against guns', 'Makes no sense']
features_train = training[labels].values
features_test = testing[labels].values
tweet_train = training['Tweet']
tweet_test = testing['Tweet']

In [43]:
max_words = 20000
chunk_size = 200

In [44]:
# Converting Tweets to Embeddings
def convert_embedding(text):
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(list(text))
    tokenized = tokenizer.texts_to_sequences(text)
    embedding = pad_sequences(tokenized, maxlen=chunk_size)   # Every Tweet is represented by vector of length 200
    return embedding

In [45]:
x_train = convert_embedding(tweet_train)
x_test = convert_embedding(tweet_test)

In [37]:
print("Building LSTM")
model = Sequential()
model.add(Embedding(max_words, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Building LSTM


In [39]:
print("Training LSTM")
model.fit(x_train, features_train, batch_size=32, epochs=20, validation_split=0.1)

Training LSTM
Train on 306 samples, validate on 34 samples
Epoch 1/20
306/306 [==============================] - 3s 11ms/step - loss: 0.5624 - acc: 0.7113 - val_loss: 0.6313 - val_acc: 0.6765
Epoch 2/20
306/306 [==============================] - 3s 11ms/step - loss: 0.5186 - acc: 0.7702 - val_loss: 0.6170 - val_acc: 0.6667
Epoch 3/20
306/306 [==============================] - 3s 11ms/step - loss: 0.4540 - acc: 0.7930 - val_loss: 0.6201 - val_acc: 0.6275
Epoch 4/20
306/306 [==============================] - 3s 11ms/step - loss: 0.3719 - acc: 0.8453 - val_loss: 0.6098 - val_acc: 0.6373
Epoch 5/20
306/306 [==============================] - 3s 11ms/step - loss: 0.2915 - acc: 0.8932 - val_loss: 0.6875 - val_acc: 0.6667
Epoch 6/20
306/306 [==============================] - 3s 11ms/step - loss: 0.2221 - acc: 0.9172 - val_loss: 0.7827 - val_acc: 0.6765
Epoch 7/20
306/306 [==============================] - 3s 11ms/step - loss: 0.1585 - acc: 0.9499 - val_loss: 0.6779 - val_acc: 0.7353
Epoch 8/20

In [47]:
inp = Input(shape=(chunk_size, ))
embed_size = 128
x1 = Embedding(max_words, embed_size)(inp)
x1 = LSTM(60, return_sequences=True, name='lstm_layer')(x1)
x1 = GlobalMaxPool1D()(x1)
# x1 = Dropout(0.1)(x1)
x1 = Dense(50, activation='relu')(x1)
x1 = Dropout(0.1)(x1)
x1 = Dense(3, activation='sigmoid')(x1)
model = Model(inputs=inp, outputs=x1)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, features_train, epochs=5, batch_size=32, validation_data=(x_test, features_test))

Train on 340 samples, validate on 81 samples
Epoch 1/5
340/340 [==============================] - 4s 11ms/step - loss: 0.6822 - acc: 0.6196 - val_loss: 0.6660 - val_acc: 0.6667
Epoch 2/5
340/340 [==============================] - 3s 8ms/step - loss: 0.6350 - acc: 0.6686 - val_loss: 0.6371 - val_acc: 0.6667
Epoch 3/5
340/340 [==============================] - 3s 8ms/step - loss: 0.6208 - acc: 0.6667 - val_loss: 0.6117 - val_acc: 0.6667
Epoch 4/5
340/340 [==============================] - 3s 8ms/step - loss: 0.6121 - acc: 0.6667 - val_loss: 0.6029 - val_acc: 0.6667
Epoch 5/5
340/340 [==============================] - 3s 8ms/step - loss: 0.5970 - acc: 0.6676 - val_loss: 0.6005 - val_acc: 0.6667
